# Live-Automated Stocks Transactions on RELIANCE

## Importing some Essential Modules to help our code function

In [ ]:
import csv
import time

In [ ]:
interval_time = 0.2

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import time
import ipympl
from IPython.display import display, clear_output
%matplotlib notebook
interval = 1

#initialising the columns of our dataframe
data = pd.DataFrame([])
columns=['Timestamp','last_price','volume','scrip_id']   

## Getting the live values from a CSV file that is getting updated live

In [ ]:
#GETTING VALUES FROM LIVE CSV FILE
def get_from_csv():
    """Returns the last value added to the CSV file as a pandas DataFrame."""
    return pd.read_csv('streaming_stocks_data.csv', names=columns, index_col=['Timestamp']).tail(1)

## Calculating the value of Brokerage for the live data

In [ ]:
#BROKERAGE FUNCTION
import json
brokerage_file="BROKERAGES.json"
with open(brokerage_file) as source:
    BROKERAGES = json.load(source)
    
def calculate_brokerage(side, txn_type, price, broker):
    current_brokerages_dict=BROKERAGES[broker][txn_type]
    brokerages = 0
    for head, value in current_brokerages_dict[side].items():
        brokerages += price*value
    return brokerages   

## Calcualting the Slope of the Closing Price of the stock

In [ ]:
#SLOPE FUNCTION
import math
def calc_slope(x):
    slope = np.polyfit(range(len(x)), x, 1)[0]
    return math.degrees(math.atan(slope))

## Main Magic function that predicts Buy/Sell and calculates all desired values for our transaction table

In [ ]:
#FUNCTION FOR BUY/SELL , QUANTITY , PROFIT , BROKERAGE , PAB , CUM_PAB

#variable intitialisations
import math
last_action=-3
last_sell=0
last_pab=0
date_buy=44
x_bar=[]
y_bar=[]
loss_no=0
app_no=0

#-----------------------------------------------------------------------------------------------------------------------

#buysell function

def buysell(count,data):
    global last_action
    global last_sell
    global last_pab
    global cum_array
    global x_bar
    global y_bar
    global loss_no
    global app_no
    if(count>9):
        
        #at 3:10pm it does a force buy/sell if cycle is still open      
        if((data.index[-1][11:19]=='15:10:00' or data.index[-1][11:19]=='15:11:00' or data.index[-1][11:19]=='15:12:00'or data.index[-1][11:19]=='15:13:00') and last_action==-1):
            profit=data.iloc[-1]['last_price']-last_sell
            action="buy"
            quantity=0
            last_action=quantity
            brokerage=calculate_brokerage(action,'intraday',data.iloc[-1]['last_price'],"zerodha")
            pab=profit-brokerage
            cum_pab=last_pab + pab
            last_pab =cum_pab
            return data.index[-1],quantity,action,data.iloc[-1]['last_price'],profit,brokerage,pab,cum_pab
        
        
        #at 3:29pm it does a force buy/sell if cycle is still open ( safety measure )
        elif(data.index[-1][11:19]=='15:29:00' and last_action==-1):
            profit=data.iloc[-1]['last_price']-last_sell
            action="buy"
            quantity=0
            last_action=quantity
            brokerage=calculate_brokerage(action,'intraday',data.iloc[-1]['last_price'],"zerodha")
            pab=profit-brokerage
            cum_pab=last_pab + pab
            y_bar.append(cum_pab)
            x_bar.append(data.index[-1][0:10])
            last_pab =0
            loss_no=0
            return data.index[-1],quantity,action,data.iloc[-1]['last_price'],profit,brokerage,pab,cum_pab
        
        if((data.index[-1][11:19]=='15:10:00' or data.index[-1][11:19]=='15:11:00' or data.index[-1][11:19]=='15:12:00' or data.index[-1][11:19]=='15:13:00') and last_action==1 ):
            profit=data.iloc[-1]['last_price']-last_sell
            action="sell"
            quantity=0
            last_action=quantity
            brokerage=calculate_brokerage(action,'intraday',data.iloc[-1]['last_price'],"zerodha")
            pab=profit-brokerage
            cum_pab=last_pab + pab
            last_pab =cum_pab
            return data.index[-1],quantity,action,data.iloc[-1]['last_price'],profit,brokerage,pab,cum_pab
        
        elif(data.index[-1][11:19]=='15:29:00' and last_action==1 ):
            profit=data.iloc[-1]['last_price']-last_sell
            action="sell"
            quantity=0
            last_action=quantity
            brokerage=calculate_brokerage(action,'intraday',data.iloc[-1]['last_price'],"zerodha")
            pab=profit-brokerage
            cum_pab=last_pab + pab
            x_bar.append(data.index[-1][0:10])
            y_bar.append(cum_pab)
            last_pab =0
            loss_no=0
            return data.index[-1],quantity,action,data.iloc[-1]['last_price'],profit,brokerage,pab,cum_pab
        
        elif(data.index[-1][11:19]=='15:27:00' or data.index[-1][11:19]=='15:28:00' or data.index[-1][11:19]=='15:29:00'):
            if(app_no==0):
                x_bar.append(data.index[-1][0:10])
                y_bar.append(last_pab)
                last_pab=0
                loss_no=0
                app_no=1
                return
            
        if(data.index[-1][11:19] >='15:11:00'):
            return
        if(data.index[-1][11:19] >='09:15:00' and data.index[-1][11:19] < '09:32:00' ):
            app_no=0
            return

#-----------------------------------------------------------------------------------------------------------------------
        
        #BUY CYCLES 
        if((data.iloc[-1]['last_price']<data.iloc[-1]['lower_b'])):
            #print((((data.iloc[-1]['last_price']-data.iloc[-1]['MA_9'])/data.iloc[-1]['last_price'])*100))
             
            if(loss_no==1):
                return    
            #OPENENING CYCLE WITHOUT ANY PREVIOUS TRANSACTION ( similar to openening cycle at quantity=0 )
            if(last_action==-3):
                action="buy"
                quantity=1
                last_action=quantity
                profit=0
                last_sell=data.iloc[-1]['last_price']
                #calculating brokerage
                brokerage=calculate_brokerage(action,'intraday',data.iloc[-1]['last_price'],"zerodha")
                #calculating pab
                pab=profit-brokerage
                #calculating cum_pab
                cum_pab=last_pab + pab
                last_pab =cum_pab
                return data.index[-1],quantity,action,data.iloc[-1]['last_price'],profit,brokerage,pab,cum_pab
            
            #IF QUANTITY IS ALREADY 1 - skip buy transaction
            elif(last_action==1):
                return 
            
            #OPENEING OF CYCLE AT QUANTITY=0
            elif(last_action==0):
                action="buy"
                quantity=1
                last_action=quantity
                profit=0
                last_sell=data.iloc[-1]['last_price']
                brokerage=calculate_brokerage(action,'intraday',data.iloc[-1]['last_price'],"zerodha")
                pab=profit-brokerage
                cum_pab=last_pab + pab
                last_pab =cum_pab
                return data.index[-1],quantity,action,data.iloc[-1]['last_price'],profit,brokerage,pab,cum_pab
            
            #BUY ACTION WHEN SELL=-1
            elif(last_action==-1):
                #calculating profit
                profit=data.iloc[-1]['last_price']-last_sell
                if(profit>0):
                    action="buy"
                    quantity=0
                    last_action=quantity
                    brokerage=calculate_brokerage(action,'intraday',data.iloc[-1]['last_price'],"zerodha")
                    pab=profit-brokerage
                    cum_pab=last_pab + pab
                    last_pab =cum_pab
                    return data.index[-1],quantity,action,data.iloc[-1]['last_price'],profit,brokerage,pab,cum_pab
                
                   
                #if time higher than 3:05pm , it allows loss till 1.5 rupees
                if(data.index[-1][11:19] >='15:05:00'):
                    profit=data.iloc[-1]['last_price']-last_sell
                    if(profit>-1.5):
                        if(loss_no==1):
                            return
                        else:
                            loss_no=1
                            action="buy"
                            quantity=0
                            last_action=quantity
                            brokerage=calculate_brokerage(action,'intraday',data.iloc[-1]['last_price'],"zerodha")
                            pab=profit-brokerage
                            cum_pab=last_pab + pab
                            last_pab =cum_pab
                            return data.index[-1],quantity,action,data.iloc[-1]['last_price'],profit,brokerage,pab,cum_pab
                    else:
                        return
                    
                #if time higher than 3:00pm , it allows loss till 1 rupees
                if(data.index[-1][11:19] >='15:00:00'):
                    profit=data.iloc[-1]['last_price']-last_sell
                    if(profit>-1):
                        if(loss_no==1):
                            return
                        else:
                            loss_no=1
                            action="buy"
                            quantity=0
                            last_action=quantity
                            brokerage=calculate_brokerage(action,'intraday',data.iloc[-1]['last_price'],"zerodha")
                            pab=profit-brokerage
                            cum_pab=last_pab + pab
                            last_pab =cum_pab
                            return data.index[-1],quantity,action,data.iloc[-1]['last_price'],profit,brokerage,pab,cum_pab
                    else:
                        return    
                    
                #if time higher than 2:45pm , it allows loss till 0.5 rupees
                if(data.index[-1][11:19] >='14:45:00'):
                    profit=data.iloc[-1]['last_price']-last_sell
                    if(profit>-0.5):
                        if(loss_no==1):
                            return
                        else:
                            loss_no=1
                            action="buy"
                            quantity=0
                            last_action=quantity
                            brokerage=calculate_brokerage(action,'intraday',data.iloc[-1]['last_price'],"zerodha")
                            pab=profit-brokerage
                            cum_pab=last_pab + pab
                            last_pab =cum_pab
                            return data.index[-1],quantity,action,data.iloc[-1]['last_price'],profit,brokerage,pab,cum_pab
                    else:
                        return  
                   
                else: 
                    return    

#-----------------------------------------------------------------------------------------------------------------------                
                
        #SELL CYCLES
        elif((data.iloc[-1]['last_price']>data.iloc[-1]['upper_b'])): 
            #print((((data.iloc[-1]['last_price']-data.iloc[-1]['MA_9'])/data.iloc[-1]['last_price'])*100))
            if(loss_no==1):
                return
            #OPENENING CYCLE WITHOUT ANY PREVIOUS TRANSACTION ( similar to openening cycle at quantity=0 )
            if(last_action==-3):
                action="sell"
                quantity=-1
                last_action=quantity
                last_sell=data.iloc[-1]['last_price']
                profit=0
                brokerage=calculate_brokerage(action,'intraday',data.iloc[-1]['last_price'],"zerodha")
                pab=profit-brokerage
                cum_pab=last_pab + pab
                last_pab =cum_pab
                return data.index[-1],quantity,action,data.iloc[-1]['last_price'],profit,brokerage,pab,cum_pab

            #IF QUANTITY IS ALREADY -1 - skip SELL transaction
            elif(last_action==-1):
                return  
            
            #OPENEING OF CYCLE AT QUANTITY=0
            elif(last_action==0):
                action="sell"
                quantity=-1
                last_action=quantity
                last_sell=data.iloc[-1]['last_price']
                profit=0
                brokerage=calculate_brokerage(action,'intraday',data.iloc[-1]['last_price'],"zerodha")
                pab=profit-brokerage
                cum_pab=last_pab + pab
                last_pab =cum_pab
                return data.index[-1],quantity,action,data.iloc[-1]['last_price'],profit,brokerage,pab,cum_pab
            
            #SELL ACTION WHEN BUY=1
            elif(last_action==1):
                profit=data.iloc[-1]['last_price']-last_sell
                if(profit>0):
                    action="sell"
                    quantity=0
                    last_action=quantity
                    brokerage=calculate_brokerage(action,'intraday',data.iloc[-1]['last_price'],"zerodha")
                    pab=profit-brokerage
                    cum_pab=last_pab + pab
                    last_pab =cum_pab
                    return data.index[-1],quantity,action,data.iloc[-1]['last_price'],profit,brokerage,pab,cum_pab
                
                if(data.index[-1][11:19] >='15:05:00'):
                    #print(data.index[-1][11:19])
                    profit=data.iloc[-1]['last_price']-last_sell
                    if(profit>-1.5):
                        if(loss_no==1):
                            return
                        else:
                            loss_no=1
                            action="sell"
                            quantity=0
                            last_action=quantity
                            brokerage=calculate_brokerage(action,'intraday',data.iloc[-1]['last_price'],"zerodha")
                            pab=profit-brokerage
                            cum_pab=last_pab + pab
                            last_pab =cum_pab
                            return data.index[-1],quantity,action,data.iloc[-1]['last_price'],profit,brokerage,pab,cum_pab
                    else:
                        return
                    
                if(data.index[-1][11:19] >='15:00:00'):
                    #print(data.index[-1][11:19])
                    profit=data.iloc[-1]['last_price']-last_sell
                    if(profit>-1.0):
                        if(loss_no==1):
                            return
                        else:
                            loss_no=1
                            action="sell"
                            quantity=0
                            last_action=quantity
                            brokerage=calculate_brokerage(action,'intraday',data.iloc[-1]['last_price'],"zerodha")
                            pab=profit-brokerage
                            cum_pab=last_pab + pab
                            last_pab =cum_pab
                            return data.index[-1],quantity,action,data.iloc[-1]['last_price'],profit,brokerage,pab,cum_pab
                    else:
                        return     
            
                if(data.index[-1][11:19] >='14:45:00'):
                    #print(data.index[-1][11:19])
                    profit=data.iloc[-1]['last_price']-last_sell
                    if(profit>-0.5):
                        if(loss_no==1):
                            return
                        else:
                            loss_no=1
                            action="sell"
                            quantity=0
                            last_action=quantity
                            brokerage=calculate_brokerage(action,'intraday',data.iloc[-1]['last_price'],"zerodha")
                            pab=profit-brokerage
                            cum_pab=last_pab + pab
                            last_pab =cum_pab
                            return data.index[-1],quantity,action,data.iloc[-1]['last_price'],profit,brokerage,pab,cum_pab
                    else:
                        return 
                else:
                    return                 

## Block of code that creates the live transactions table on the incoming live data

In [ ]:
#BASIC IMPORTS
import numpy as np
import plotly.express as px
import plotly.offline as pyo
import plotly.graph_objs as go

#-----------------------------------------------------------------------------------

#CREATING THE DATAFRAMES IN WHICH WE WILL BE WORKING ON

#dataframe containing all the live data 
data=pd.DataFrame(columns=columns,index=['Timestamp'])

#dataframe containing only the buy/sell rows
acdata=pd.DataFrame(columns=['timestamp','quantity','action','last_price','profit','brokerage','pab','cum_pab'])

data['20_SMA']=np.nan
data['upper_b']=np.nan
data['lower_b']=np.nan
#data['slope']=np.nan
data.drop('Timestamp',axis=1,inplace=True)
data.dropna(inplace=True)

count=0
'''x=[]
#ym=[]
y=[]'''

#-----------------------------------------------------------------------------------
while True:
    
    #ADDING THE DATA INTO OUR DATAFRAME
    data=data.append(get_from_csv())
    data= data[['scrip_id','volume','last_price','20_SMA','lower_b','upper_b']]
    time.sleep(interval_time)
    
    def write_file(timestamp, close_price):
        file = open("sampleText.txt","a")
        l = [timestamp,",",str(close_price),"\n"]
        file.writelines(l)
        file.close()
    write_file(data.index[-1],data.iloc[-1]['last_price'])    
    #-------------------------------------------------------------------------------
    #ani = animation.FuncAnimation(plt.gcf(), animate, interval=1000)
    #CLEARING and PRINTING OUTPUT AFTER EVERY PRINT
    clear_output(wait=True)
    print("GENERAL TABLE \n")
    print("--------------------------------------------------------------------------------")
    with pd.option_context('expand_frame_repr', False):
        print(data.tail(10))
    
    
    #fig.canvas.draw()    
    print("\n")
    print("ACTION TABLE \n")
    print("--------------------------------------------------------------------------------")
    
    #-------------------------------------------------------------------------------
    
    #CALCULATING 20 SIMPLE MINUTE MOVING AVERAGE
    data['20_SMA']=data['last_price'].rolling(12).mean()
    
    #-------------------------------------------------------------------------------
    
    #CALCULATING UPPER BOLLINGER BAND
    data['upper_b']=data['20_SMA'] + 2*(data['last_price'].rolling(20).std())
    
    #-------------------------------------------------------------------------------
    
    #CALCULATING LOWER BOLLINGER BAND
    data['lower_b']=data['20_SMA'] - 2*(data['last_price'].rolling(20).std())
    
    #-------------------------------------------------------------------------------
    
    #PLOTTING 9 MINUTE MOVING AVERAGE AND LAST PRICE AFTER EVERY 20 MINUTES    
    count=len(data)
    #priceplot(count,data)
    
    #-------------------------------------------------------------------------------
    
    #PREDICTING BUY/SELL and Quantity
    
    # RETURNS timestamp,quantity , action , last price , profit in a tuple
    if(count>19):
        #ax.plot(x[-30:], ym[-30:], color='r')
        decision=(buysell(count,data))
    
        if decision is None:
            pass
        else:
            #appending to new dataframe
            acdata=acdata.append(pd.Series(decision,index=acdata.columns),ignore_index=True)
    if(acdata.empty):
        print("No elements present yet !")
    else:    
        with pd.option_context('expand_frame_repr', False):
            print(acdata.tail(10))
    
    #--------------------------------------------------------------------------------

## Few essential imports for helping us perform interative plots

In [ ]:
%matplotlib notebook
import matplotlib.pyplot as plt

## Plots the last 700 values of the trends of the Close Price and the upper and lower Bollinger Bands of RELIANCE

In [ ]:
data[['last_price','20_SMA','lower_b','upper_b']].tail(700).plot(figsize=(10,6))

## Plotting transaction Tickers that help us to observe where exactly we bought/sold 

In [ ]:
data.index = pd.to_datetime(data.index)
acdata1=acdata.copy(deep=True)
acdata1=acdata1.set_index('timestamp')
acdata1.index = pd.to_datetime(acdata1.index)

In [ ]:
fig, ax = plt.subplots(figsize=(15,6))
plt.xticks(rotation=90)
buy = acdata1['action'] == 'buy'
sell = acdata1['action'] == 'sell'

ax.clear()
ax.plot(data.index,data['last_price'], label='Close Price')
#ax.plot(data.index,data['20_SMA'], label='Close Price')
ax.plot(data.index,data['upper_b'], label='Close Price')
ax.plot(data.index,data['lower_b'], label='Close Price')
#     ax.plot(data_df.index.astype(str).str.slice(start=11, stop=-6),
#             data_df[f'MA_{sma_win}'], label='Moving Average')

    # Plotting data
ax.scatter(x=acdata1[buy].index,
               y=acdata1[buy]['last_price'],
               color='green', label='Buy', marker='^',s=60)
ax.scatter(x=acdata1[sell].index,
               y=acdata1[sell]['last_price'],
               color='red', label='Sell', marker='v',
               s=60)

## Plots day wise Cummulative Profit after Brokerages for multiple days that the model has been run for

In [ ]:
fig, ax = plt.subplots(figsize=(9,6))
ax.bar(x=x_bar, height=y_bar)
plt.xticks(rotation=90)